# Reinforcement Learning Simulation in OpenAI Gym Environment

## 1. CartPole-v1

In [1]:
import gym
import random

env = gym.make("CartPole-v1")
print(env.observation_space)
print(env.action_space)

Box(4,)
Discrete(2)


In [5]:
class Agent():
    def __init__(self, env):
        self.action_size = env.action_space.n
        self.obs = env.observation_space
        self.actions = env.action_space
        print("||Action-Space|| = {}".format(self.action_size))
        
    def get_action(self, state):
        #action = random.choice(range(self.action_size))
        pole_angle = state[2]
        action = 0 if pole_angle < 0 else 1
        return action

In [8]:
myagent = Agent(env)

n = int(input("Enter the no. of episodes for simulation: "))

for episode in range(n):
    state = env.reset()
    for t in range(10000):
        env.render()
        print("In State: ", state)
        action = myagent.get_action(state)
        state, reward, done, info = env.step(action)
        if done:
            print("Episode {} done in {} timesteps!\n".format(episode+1, t+1))
            break
    
env.close()   

||Action-Space|| = 2
Enter the no. of episodes for simulation: 5
In State:  [-0.04385015 -0.02910051 -0.02935307  0.02883278]
In State:  [-0.04443216 -0.22378949 -0.02877641  0.31211189]
In State:  [-0.04890795 -0.41848991 -0.02253417  0.59558252]
In State:  [-0.05727775 -0.61328936 -0.01062252  0.88108302]
In State:  [-0.06954354 -0.80826542  0.00699914  1.17040764]
In State:  [-0.08570885 -0.61323519  0.03040729  0.87992719]
In State:  [-0.09797355 -0.41853925  0.04800584  0.59695666]
In State:  [-0.10634434 -0.22412079  0.05994497  0.31977324]
In State:  [-0.11082675 -0.02990151  0.06634043  0.04658086]
In State:  [-0.11142478  0.16420953  0.06727205 -0.22445544]
In State:  [-0.10814059  0.35830874  0.06278294 -0.49518263]
In State:  [-0.10097442  0.55249173  0.05287929 -0.76743719]
In State:  [-0.08992458  0.74684737  0.03753055 -1.04302388]
In State:  [-0.07498763  0.94145136  0.01667007 -1.32369292]
In State:  [-0.05615861  1.13635882 -0.00980379 -1.61111271]
In State:  [-0.03343

In State:  [ 0.16074742  0.59447883 -0.19844957 -0.88500528]
Episode 4 done in 34 timesteps!

In State:  [-0.02449188  0.00095633  0.02391231 -0.03665428]
In State:  [-0.02447275  0.19572736  0.02317922 -0.32169775]
In State:  [-0.0205582   0.39051169  0.01674527 -0.60698168]
In State:  [-0.01274797  0.58539555  0.00460563 -0.89434361]
In State:  [-1.04005980e-03  7.80454744e-01 -1.32812389e-02 -1.18557525e+00]
In State:  [ 0.01456904  0.58550755 -0.03699274 -0.89708481]
In State:  [ 0.02627919  0.39090608 -0.05493444 -0.6162557 ]
In State:  [ 0.03409731  0.19659291 -0.06725955 -0.34136823]
In State:  [ 0.03802917  0.00248916 -0.07408692 -0.07063034]
In State:  [ 0.03807895 -0.19149675 -0.07549953  0.19778869]
In State:  [ 0.03424901 -0.38546217 -0.07154375  0.4657322 ]
In State:  [ 0.02653977 -0.57950415 -0.06222911  0.73503467]
In State:  [ 0.01494969 -0.77371377 -0.04752841  1.00750167]
In State:  [-5.24587935e-04 -9.68170055e-01 -2.73783808e-02  1.28488862e+00]
In State:  [-0.01988